In [ ]:
import numpy as np
import matplotlib.pyplot as plt

DATAMAX_ = 10000

oneday_ = 60 * 60 * 24

np.random.seed(seed=120)

t_start = 1e3 # first measurement time
time_fin = 100.0 * oneday_ # last measurement time

# Swift instrument dependent properties
time_orb = 2880 # time of orbit
count_lim = 1e-4 #  // sensitivity limit for Swift
time_cut = 1.0 * oneday_ # ; // time after which only fractional exposure
frac_expo = 0.1 # // fraction of exposure
bincountsbase = 20 #; //counts per bin for 1 ct / s
onecount = 3.8e-11 #; // one count is this much erg cm-2
deltanu = 2.38e18 #; // 0.3 - 10 KeV - translated to Hertz

################################################################################
# an example function, using some global variables

def F(t):
  scale = 0.01
  
  if (t < t_break):
    return scale * (t / t_break)**(alpha_1)
  elif (t < t_break2):
    return scale * (t / t_break)**(alpha_2)
  else:
    return scale * (t/t_break2)**(alpha_3)

################################################################################

def makedata(time, rate, synth, err, synth_err):
  i = 0
  time[0] = t_start
  time_cur = t_start

  # This while loop is basically C-programming, not python. It is therefore very
  # slow, and might end up an unacceptable bottle-neck if many light curves
  # are generated
  while True:
    n_local = i
    rate[i] = F(time[i])
     
    # set time of next datapoint, rought estimate by jumping forward in time
    # assuming current rate to be representative until next time entry
    if (time[i] < time_cut): # early time, full exposure
      time[i + 1] = time[i] + bincountsbase / rate[i]
    else: # late time, only fractional exposure
      time[i + 1] = time[i] + bincountsbase / frac_expo / rate[i]

    # implement orbital time loss. It increasing the jump in time, because
    # no new photons are added during the time loss   
    if (time[i+1] > time_cur + time_orb and time[i+1] < time_cur + 2*time_orb):
      time[i+1] = time_cur + 2 * time_orb
      time_cur = time[i + 1]
 
    # check if we either have the full requested time span or the signal
    #/ has dropped below the detection threshold. In both cases, bail out.
    if (time[i] > time_fin):
      break
    if (rate[i] < count_lim):
      if (rate[i] < 0.5 * count_lim):
        i = i - 1
        n_local = n_local - 1
      break 

    i = i+1

  # should not happen, unless your count rate function is so high that you
  # generate way too many data points.
  if (n_local >= DATAMAX_):
    print ("too many datapoints! Lower your normalization.")
    exit()

  # set the errors on the data points. You can experiment with different
  # approaches to the errors here.
  # This is 'proper python' where we set all entries in one go
  err[0:n_local] = np.where(rate[0:n_local] >= 1e-3, rate[0:n_local] * 0.25,
    rate[0:n_local] * 0.25 * 2)
  
  #print( rate[0:n_local] )# some debugging info
  #print err[0:n_local] # some debugging info
  deviate = np.random.normal(size=n_local) * err[0:n_local]
  #print deviate # some debugging info

  # now create perturbed data points, using the errors as computed above
  # on the unperturbed data points as computed above  
  synth[0:n_local] = rate[0:n_local] + deviate
  synth_err[0:n_local] = np.where(synth[0:n_local] >= 1e-3, 
    synth[0:n_local] * 0.25, synth[0:n_local] * 0.25 * 2)

  return n_local

##################

# prep a light curve
alpha_1 = -2.1 + np.random.normal(loc=0., scale=0.3)
alpha_2 = alpha_1 + 1.5
t_break = 0.5 * oneday_ + np.random.normal(loc = 0., scale = 0.1)
time = np.zeros(DATAMAX_)
rate = np.zeros(DATAMAX_)
synth = np.zeros(DATAMAX_)
err = np.zeros(DATAMAX_)
synth_err = np.zeros(DATAMAX_)

#n = makedata(time, rate, synth, err, synth_err)

# prep another one

alpha_1 = -2.1 + np.random.normal(loc=0., scale=0.1)
t_break = 0.1 * oneday_ + np.random.normal(loc = 0., scale = 0.1)
t_break2 = 3*oneday_
alpha_2 = alpha_1 +2
alpha_3 = alpha_1 - 0.5
time2 = np.zeros(DATAMAX_)
rate2 = np.zeros(DATAMAX_)
synth2 = np.zeros(DATAMAX_)
err2 = np.zeros(DATAMAX_)
synth_err2 = np.zeros(DATAMAX_)

n2 = makedata(time2, rate2, synth2, err2, synth_err2)

# plot both examples

plt.loglog(time, rate,'r')
plt.errorbar(time, synth, synth_err, 0, 'r.')

plt.loglog(time2, rate2, 'g')
plt.errorbar(time2, synth2, synth_err2, 0, 'g.')
plt.draw()
plt.show()

In [ ]:
#power law starting with shallow then steep, plotted like curves
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from scipy.stats import kde
import os

k=0
for k in range(0,2000):
    DATAMAX_ = 10000

    oneday_ = 60 * 60 * 24.

    t_start = 1e3 # first measurement time
    time_fin = 100.0 * oneday_ # last measurement time

    # Swift instrument dependent properties
    time_orb = 2880 # time of orbit
    count_lim = 1e-4 #  // sensitivity limit for Swift
    time_cut = 1.0 * oneday_ # ; // time after which only fractional exposure
    frac_expo = 0.1 # // fraction of exposure
    bincountsbase = 20 #; //counts per bin for 1 ct / s
    onecount = 3.8e-11 #; // one count is this much erg cm-2
    deltanu = 2.38e18 #; // 0.3 - 10 KeV - translated to Hertz

    ################################################################################
    # an example function, using some global variables

    alpha_1 = -2.1
    t_break = 0.5 * oneday_
    alpha_2 = alpha_1 -1.5

    def F(t):
      scale = 0.001

      if (t < t_break):
        return scale * (t / t_break)**(alpha_1)
      else:
        return scale * (t / t_break)**(alpha_2)

    ################################################################################

    def makedata(time, rate, synth, err, synth_err):
      i = 0
      time[0] = t_start
      time_cur = t_start

      # This while loop is basically C-programming, not python. It is therefore very
      # slow, and might end up an unacceptable bottle-neck if many light curves
      # are generated
      while True:
        n_local = i
        rate[i] = F(time[i])

        # set time of next datapoint, rought estimate by jumping forward in time
        # assuming current rate to be representative until next time entry
        if (time[i] < time_cut): # early time, full exposure
          time[i + 1] = time[i] + bincountsbase / rate[i]
        else: # late time, only fractional exposure
          time[i + 1] = time[i] + bincountsbase / frac_expo / rate[i]

        # implement orbital time loss. It increasing the jump in time, because
        # no new photons are added during the time loss   
        if (time[i+1] > time_cur + time_orb and time[i+1] < time_cur + 2*time_orb):
          time[i+1] = time_cur + 2 * time_orb
          time_cur = time[i + 1]

        # check if we either have the full requested time span or the signal
        #/ has dropped below the detection threshold. In both cases, bail out.
        if (time[i] > time_fin):
          break
        if (rate[i] < count_lim):
          if (rate[i] < 0.5 * count_lim):
            i = i - 1
            n_local = n_local - 1
          break 

        i = i+1

      # should not happen, unless your count rate function is so high that you
      # generate way too many data points.
      if (n_local >= DATAMAX_):
        print ("too many datapoints! Lower your normalization.")
        exit()

      # set the errors on the data points. You can experiment with different
      # approaches to the errors here.
      # This is 'proper python' where we set all entries in one go
      err[0:n_local] = np.where(rate[0:n_local] >= 1e-3, rate[0:n_local] * 0.25,
        rate[0:n_local] * 0.25 * 2)

      #print( rate[0:n_local] )# some debugging info
      #print err[0:n_local] # some debugging info
      deviate = np.random.normal(size=n_local) * err[0:n_local]
      #print deviate # some debugging info

      # now create perturbed data points, using the errors as computed above
      # on the unperturbed data points as computed above  
      synth[0:n_local] = rate[0:n_local] + deviate
      synth_err[0:n_local] = np.where(synth[0:n_local] >= 1e-3, 
        synth[0:n_local] * 0.25, synth[0:n_local] * 0.25 * 2)

      return n_local

    ##################

    # prep a light curve
    alpha_1 = -1.1 + np.random.normal(loc=0., scale=0.3)
    alpha_2 = alpha_1 - 1.5
    t_break = 0.5 * oneday_ + np.random.normal(loc = 0., scale = 0.1)
    time = np.zeros(DATAMAX_)
    rate = np.zeros(DATAMAX_)
    synth = np.zeros(DATAMAX_)
    err = np.zeros(DATAMAX_)
    synth_err = np.zeros(DATAMAX_)

    n = makedata(time, rate, synth, err, synth_err)
    
    time1 = time[time!=0]
    synth1 = synth[synth>0]
    synth_err1 = synth_err[synth_err!=0]
    synth_err1 = synth_err1[0:len(synth1)]
    time1 = time1[0:len(synth1)]
    
    x = np.log(time1)
    #print(len(x))
    #print(x)
    y = np.log(synth1)
    #print(len(y))
    #print(y)
    err = synth_err1/synth1
    #print(len(err))
    #print(err)

    #use average errors, interpolate and generate random points
    x_avg_err = np.median(err)
    y_avg_err = np.median(err)

    #set limits on axes so the aspect ratio is a constant - no stretching
    #constant = 1
    tmax = x.max()
    tmin = x.min()
    fmin = y.min() 
    fmax = 2*(tmax-tmin)+fmin

    if len(x)>4:    
        xnew = np.linspace(min(x), max(x), num=1000, endpoint=True)
        f2 = interp1d(x, y, kind='linear')
        ynew = f2(xnew)
        xnp = np.array(xnew)
        ynp = np.array(ynew)

        xpoints = []
        ypoints = []
        for i in range(len(xnp)):
            xpoints.append(np.random.normal(loc=xnp[i],scale=x_avg_err, size=(10000)))
            ypoints.append(np.random.normal(loc=ynp[i],scale=y_avg_err, size=(10000)))
        xpoints = np.array(xpoints)
        ypoints = np.array(ypoints)
        xpoints=xpoints.reshape(10000000)
        ypoints=ypoints.reshape(10000000)

        #plot - setting axes limits
        plot = plt.figure()
        plot = plt.hist2d(xpoints, ypoints, bins=40, cmap='binary')
        plot = plt.xlim(tmin, tmax)
        plot = plt.ylim(fmin, fmax)
        plot = plt.axis('off')
        #plot = plt.show()
        plot = plt.savefig('fake_data1/type1/l_{0}.png'.format(k))
    k=k+1

In [ ]:
#power law starting with steep then shallow
#power law starting with shallow then steep, plotted like curves
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from scipy.stats import kde
import os

k=0
for k in range(0,2000):
    DATAMAX_ = 1000

    oneday_ = 60 * 60 * 24.

    t_start = 1e3 # first measurement time
    time_fin = 100.0 * oneday_ # last measurement time

    # Swift instrument dependent properties
    time_orb = 2880 # time of orbit
    count_lim = 1e-4 #  // sensitivity limit for Swift
    time_cut = 1.0 * oneday_ # ; // time after which only fractional exposure
    frac_expo = 0.1 # // fraction of exposure
    bincountsbase = 20 #; //counts per bin for 1 ct / s
    onecount = 3.8e-11 #; // one count is this much erg cm-2
    deltanu = 2.38e18 #; // 0.3 - 10 KeV - translated to Hertz

    ################################################################################
    # an example function, using some global variables

    alpha_1 = -2.1
    t_break = 0.5 * oneday_
    alpha_2 = alpha_1 -1.5

    def F(t):
      scale = 0.001

      if (t < t_break):
        return scale * (t / t_break)**(alpha_1)
      else:
        return scale * (t / t_break)**(alpha_2)

    ################################################################################

    def makedata(time, rate, synth, err, synth_err):
      i = 0
      time[0] = t_start
      time_cur = t_start

      # This while loop is basically C-programming, not python. It is therefore very
      # slow, and might end up an unacceptable bottle-neck if many light curves
      # are generated
      while True:
        n_local = i
        rate[i] = F(time[i])

        # set time of next datapoint, rought estimate by jumping forward in time
        # assuming current rate to be representative until next time entry
        if (time[i] < time_cut): # early time, full exposure
          time[i + 1] = time[i] + bincountsbase / rate[i]
        else: # late time, only fractional exposure
          time[i + 1] = time[i] + bincountsbase / frac_expo / rate[i]

        # implement orbital time loss. It increasing the jump in time, because
        # no new photons are added during the time loss   
        if (time[i+1] > time_cur + time_orb and time[i+1] < time_cur + 2*time_orb):
          time[i+1] = time_cur + 2 * time_orb
          time_cur = time[i + 1]

        # check if we either have the full requested time span or the signal
        #/ has dropped below the detection threshold. In both cases, bail out.
        if (time[i] > time_fin):
          break
        if (rate[i] < count_lim):
          if (rate[i] < 0.5 * count_lim):
            i = i - 1
            n_local = n_local - 1
          break 

        i = i+1

      # should not happen, unless your count rate function is so high that you
      # generate way too many data points.
      if (n_local >= DATAMAX_):
        print ("too many datapoints! Lower your normalization.")
        exit()

      # set the errors on the data points. You can experiment with different
      # approaches to the errors here.
      # This is 'proper python' where we set all entries in one go
      err[0:n_local] = np.where(rate[0:n_local] >= 1e-3, rate[0:n_local] * 0.25,
        rate[0:n_local] * 0.25 * 2)

      #print( rate[0:n_local] )# some debugging info
      #print err[0:n_local] # some debugging info
      deviate = np.random.normal(size=n_local) * err[0:n_local]
      #print deviate # some debugging info

      # now create perturbed data points, using the errors as computed above
      # on the unperturbed data points as computed above  
      synth[0:n_local] = rate[0:n_local] + deviate
      synth_err[0:n_local] = np.where(synth[0:n_local] >= 1e-3, 
        synth[0:n_local] * 0.25, synth[0:n_local] * 0.25 * 2)

      return n_local

    ##################

    # prep a light curve
    alpha_1 = -1.1 + np.random.normal(loc=0., scale=0.3)
    alpha_2 = alpha_1 + 0.5
    t_break = 0.5 * oneday_ + np.random.normal(loc = 0., scale = 0.1)
    time = np.zeros(DATAMAX_)
    rate = np.zeros(DATAMAX_)
    synth = np.zeros(DATAMAX_)
    err = np.zeros(DATAMAX_)
    synth_err = np.zeros(DATAMAX_)

    n = makedata(time, rate, synth, err, synth_err)
    
    time1 = time[time!=0]
    synth1 = synth[synth>0]
    synth_err1 = synth_err[synth_err!=0]
    synth_err1 = synth_err1[0:len(synth1)]
    time1 = time1[0:len(synth1)]
    
    x = np.log(time1)
    #print(len(x))
    #print(x)
    y = np.log(synth1)
    #print(len(y))
    #print(y)
    err = synth_err1/synth1
    #print(len(err))
    #print(err)

    #use average errors, interpolate and generate random points
    x_avg_err = np.median(err)
    y_avg_err = np.median(err)

    #set limits on axes so the aspect ratio is a constant - no stretching
    #constant = 1
    tmax = x.max()
    tmin = x.min()
    fmin = y.min() 
    fmax = 2*(tmax-tmin)+fmin

    if len(x)>4:    
        xnew = np.linspace(min(x), max(x), num=1000, endpoint=True)
        f2 = interp1d(x, y, kind='linear')
        ynew = f2(xnew)
        xnp = np.array(xnew)
        ynp = np.array(ynew)

        xpoints = []
        ypoints = []
        for i in range(len(xnp)):
            xpoints.append(np.random.normal(loc=xnp[i],scale=x_avg_err, size=(10000)))
            ypoints.append(np.random.normal(loc=ynp[i],scale=y_avg_err, size=(10000)))
        xpoints = np.array(xpoints)
        ypoints = np.array(ypoints)
        xpoints=xpoints.reshape(10000000)
        ypoints=ypoints.reshape(10000000)

        #plot - setting axes limits
        plot = plt.figure()
        plot = plt.hist2d(xpoints, ypoints, bins=40, cmap='binary')
        plot = plt.xlim(tmin, tmax)
        plot = plt.ylim(fmin, fmax)
        plot = plt.axis('off')
        #plot = plt.show()
        plot = plt.savefig('fake_data1/type2/l_{0}.png'.format(k+2000))
    k=k+1

In [ ]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from scipy.stats import kde
import os


k=0
for k in range(0,200):
    DATAMAX_ = 10000000

    oneday_ = 60 * 60 * 24

    t_start = 1e3 # first measurement time
    time_fin = 100.0 * oneday_ # last measurement time

    # Swift instrument dependent properties
    time_orb = 2880 # time of orbit
    count_lim = 1e-4 #  // sensitivity limit for Swift
    time_cut = 1.0 * oneday_ # ; // time after which only fractional exposure
    frac_expo = 0.1 # // fraction of exposure
    bincountsbase = 20 #; //counts per bin for 1 ct / s
    onecount = 3.8e-11 #; // one count is this much erg cm-2
    deltanu = 2.38e18 #; // 0.3 - 10 KeV - translated to Hertz

    ################################################################################
    # an example function, using some global variables


    def F(t):
        scale = 1

        if (t < t_break):
            return scale * (t / t_break)**(alpha_1)
        elif(t < t_break2):
            return scale * (t / t_break)**(alpha_2)
        else:
            scale2 = scale * (t_break2 / t_break)**(alpha_2)
            return scale2 * (t/ t_break2)**(alpha_3)
        

    ################################################################################

    def makedata(time, rate, synth, err, synth_err):
        i = 0
        time[0] = t_start
        time_cur = t_start

          # This while loop is basically C-programming, not python. It is therefore very
          # slow, and might end up an unacceptable bottle-neck if many light curves
          # are generated
        while True:
            n_local = i
            rate[i] = F(time[i])
                # set time of next datapoint, rought estimate by jumping forward in time
                # assuming current rate to be representative until next time entry
            if (time[i] < time_cut): # early time, full exposure
                time[i + 1] = time[i] + bincountsbase / rate[i]
            else: # late time, only fractional exposure
                time[i + 1] = time[i] + bincountsbase / frac_expo / rate[i]

                    # implement orbital time loss. It increasing the jump in time, because
                    # no new photons are added during the time loss   
            if (time[i+1] > time_cur + time_orb and time[i+1] < time_cur + 2*time_orb):
                time[i+1] = time_cur + 2 * time_orb
                time_cur = time[i + 1]

                    # check if we either have the full requested time span or the signal
                    #/ has dropped below the detection threshold. In both cases, bail out.
            if (time[i] > time_fin):
                break
            if (rate[i] < count_lim):
                if (rate[i] < 0.5 * count_lim):
                    i = i - 1
                    n_local = n_local - 1
                break 

            i = i+1

          # should not happen, unless your count rate function is so high that you
          # generate way too many data points.
        if (n_local >= DATAMAX_):
            print ("too many datapoints! Lower your normalization.")
            exit()

          # set the errors on the data points. You can experiment with different
          # approaches to the errors here.
          # This is 'proper python' where we set all entries in one go
        err[0:n_local] = np.where(rate[0:n_local] >= 1e-3, rate[0:n_local] * 0.25,
            rate[0:n_local] * 0.25 * 2)

        np.random.seed(seed=120)
          #print( rate[0:n_local] )# some debugging info
          #print err[0:n_local] # some debugging info
        deviate = np.random.normal(size=n_local) * err[0:n_local]
          #print deviate # some debugging info

          # now create perturbed data points, using the errors as computed above
          # on the unperturbed data points as computed above  
        synth[0:n_local] = rate[0:n_local] + deviate
        synth_err[0:n_local] = np.where(synth[0:n_local] >= 1e-3, 
            synth[0:n_local] * 0.25, synth[0:n_local] * 0.25 * 2)

        return n_local

    ##################

    np.random.seed(seed=None)
    # prep a light curve
    alpha_1 = -0.5 + np.random.uniform(low=-0.2, high=0.2)
    t_break =  10000  + np.random.uniform(low = -2000, high = 2000)#0.2* oneday_ + np.random.normal(loc = 0., scale = 0.05)
    t_break2 = 40000 + np.random.uniform(low = -2000, high = 2000)
    t_break3 = 70000 + np.random.uniform(low = -2000, high = 2000)
    alpha_2 = -3 + np.random.uniform(low=-.8, high=.8) 
    alpha_3 = -0.09 + np.random.uniform(low=-0.05, high=0.05)
    alpha_4 = -2.3 +np.random.uniform(low=-0.5, high=0.5)
    time = np.zeros(DATAMAX_)
    rate = np.zeros(DATAMAX_)
    synth = np.zeros(DATAMAX_)
    err = np.zeros(DATAMAX_)
    synth_err = np.zeros(DATAMAX_)
    
    print('alpha1 = {0}, alpha2 = {1}, tbreak = {2}'.format(alpha_1, alpha_2, t_break))

    n = makedata(time, rate, synth, err, synth_err)

    time1 = time[time!=0]
    synth1 = synth[synth>0]
    synth_err1 = synth_err[synth_err!=0]
    synth_err1 = synth_err1[0:len(synth1)]
    time1 = time1[0:len(synth1)]

    x = np.log(time1)
    print(len(x))
    #print(x)
    y = np.log(synth1)
    #print(len(y))
    #print(y)
    err = synth_err1/synth1
    #print(len(err))
    #print(err)

        #use average errors, interpolate and generate random points
    x_avg_err = np.median(err)
    y_avg_err = np.median(err)

        #set limits on axes so the aspect ratio is a constant - no stretching
        #constant = 1
    if len(x)>4:
        tmax = x.max()
        tmin = x.min()
        fmin = y.min() 
        fmax = 3*(tmax-tmin)+fmin
        xnew = np.linspace(min(x), max(x), num=1000, endpoint=True)
        f2 = interp1d(x, y, kind='linear')
        ynew = f2(xnew)
        xnp = np.array(xnew)
        ynp = np.array(ynew)

        xpoints = []
        ypoints = []
        for i in range(len(xnp)):
            xpoints.append(np.random.normal(loc=xnp[i],scale=x_avg_err, size=(10000)))
            ypoints.append(np.random.normal(loc=ynp[i],scale=y_avg_err, size=(10000)))
        xpoints = np.array(xpoints)
        ypoints = np.array(ypoints)
        xpoints=xpoints.reshape(10000000)
        ypoints=ypoints.reshape(10000000)

            #plot - setting axes limits
        plot = plt.figure()
        plot = plt.hist2d(xpoints, ypoints, bins=40, cmap='binary')
        plot = plt.xlim(tmin, tmax)
        plot = plt.ylim(fmin, fmax)
        plot = plt.axis('off')
        plot = plt.show()
    k=k+1